
# **A3.1 SVM y Multiple Testing**


**1.** Para empezar, se importan los datos, se imprime un resumen de estos y se revisa que no haya **huecos**. Para esto se usan las funciones `read_csv` y `isnull().values.any()` de la librería `pandas` que devuelve una respuesta booleana. Después, se extraen aquellas observaciones cuya **clase es 2** y aquellas cuya **clase es 4**. Con la librería `numpy` se obtiene la media de las observaciones de cada una de las clases y se obtiene la diferencia. Se imprimen los **10 genes con mayor diferencia de medias**.

In [96]:
import pandas as pd
import numpy as np

df = pd.read_csv('A3.1 Khan.csv')

print(df.head())

print("hay huecos:")
print(df.isnull().values.any())

X = df.drop(columns=['y'])
y = df['y']

X2 = X[y == 2]
X4 = X[y == 4]

media_2 = X2.mean()
media_4 = X4.mean()

diferencias = (media_2 - media_4).abs()

top_ten = diferencias.sort_values(ascending=False).head(10)
print("10 genes con mayor diferencia de medias entre la clase 2 y 4")
print(top_ten)



         X1        X2        X3        X4        X5        X6        X7  \
0  0.773344 -2.438405 -0.482562 -2.721135 -1.217058  0.827809  1.342604   
1 -0.078178 -2.415754  0.412772 -2.825146 -0.626236  0.054488  1.429498   
2 -0.084469 -1.649739 -0.241308 -2.875286 -0.889405 -0.027474  1.159300   
3  0.965614 -2.380547  0.625297 -1.741256 -0.845366  0.949687  1.093801   
4  0.075664 -1.728785  0.852626  0.272695 -1.841370  0.327936  1.251219   

         X8        X9       X10  ...     X2300     X2301     X2302     X2303  \
0  0.057042  0.133569  0.565427  ... -0.027474 -1.660205  0.588231 -0.463624   
1 -0.120249  0.456792  0.159053  ... -0.246284 -0.836325 -0.571284  0.034788   
2  0.015676  0.191942  0.496585  ...  0.024985 -1.059872 -0.403767 -0.678653   
3  0.819736 -0.284620  0.994732  ...  0.357115 -1.893128  0.255107  0.163309   
4  0.771450  0.030917  0.278313  ...  0.061753 -2.273998 -0.039365  0.368801   

      X2304     X2305     X2306     X2307     X2308  y  
0 -3.952845

En terminos de un estudio de inferencia, la diferencia de media entre clases indica si existe una correlacion significativa entre el gen y la clase de cancer. Al inferir, una mayor diferencia de medias significa que dicho gen afecta mas a si la clase es 2 o 4.

**2.** A continuación se importa la librería `ttest_ind` de `scipy.stats` para calcular el **estadístico t** y el **p-value** asociado de para cada gen cuya clase es **2 o 4**. Los estadísticos t y p-values se guardan en un vector cada uno y se transfieren a un **dataframe**. Además se importa la función `multipletests` de `statsmodels` y se obtienen los **q values** de cada gen con las metodologías: **bonferroni**, **holm** y **benjamini-hochberg**. Se agregan dichos q values al dataframe y posteriormente se separan aquellos genes que sean **significativos segun cada metología** manteniendo un control de **0.05** y se imprimen la cantidad de genes encontrados significativos y un resumen de estos.

In [97]:
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
t_stats = []
p_values = []

for col in X.columns:
  t_stat, p_val = ttest_ind(X2[col], X4[col], equal_var=False)
  t_stats.append(t_stat)
  p_values.append(p_val)

results = pd.DataFrame({'gen': X.columns, 't_stat': t_stats, 'p_value': p_values})

# bonferroni
results['p_bonferroni'] = multipletests(results['p_value'], method='bonferroni')[1]

# holm
results['p_holm'] = multipletests(results['p_value'], method='holm')[1]

# benjamini-hochberg
results['p_bh'] = multipletests(results['p_value'], method='fdr_bh')[1]

significativas_og = results[results['p_value'] < 0.05]
significativas_bonferroni = results[results['p_bonferroni'] < 0.05]
significativas_holm = results[results['p_holm'] < 0.05]
significativas_bh = results[results['p_bh'] < 0.05]

print(f"{len(significativas_og)} genes significativos originalmente:")
print(significativas_og[['gen', 'p_value']])

print(f"\n{len(significativas_bonferroni)} genes significativos con Bonferroni:")
print(significativas_bonferroni[['gen', 'p_bonferroni']])

print(f"\n{len(significativas_holm)} genes significativos con Holm:")
print(significativas_holm[['gen', 'p_holm']])

print(f"\n{len(significativas_bh)} genes significativos con Benjamini-Hochberg:")
print(significativas_bh[['gen', 'p_bh']])

547 genes significativos originalmente:
        gen       p_value
1        X2  7.383962e-08
2        X3  9.621623e-05
10      X11  4.384262e-02
25      X26  8.343076e-03
28      X29  1.113505e-03
...     ...           ...
2294  X2295  3.283459e-04
2299  X2300  2.057612e-04
2300  X2301  2.017711e-04
2301  X2302  4.626187e-02
2302  X2303  2.491232e-05

[547 rows x 2 columns]

72 genes significativos con Bonferroni:
        gen  p_bonferroni
1        X2  1.704218e-04
35      X36  1.819958e-03
66      X67  3.261383e-02
128    X129  5.808252e-06
173    X174  1.293274e-05
...     ...           ...
2045  X2046  4.083533e-11
2049  X2050  9.427801e-12
2114  X2115  1.681743e-02
2145  X2146  1.616023e-06
2246  X2247  7.076436e-04

[72 rows x 2 columns]

72 genes significativos con Holm:
        gen        p_holm
1        X2  1.682067e-04
35      X36  1.786050e-03
66      X67  3.168120e-02
128    X129  5.760437e-06
173    X174  1.282067e-05
...     ...           ...
2045  X2046  4.072917e-11
2049 

**3.** A continuación, se repite el procedimiento pero en lugar de buscar asociaciones con la clase 2 y 4, se busca comparar las medias de las **4 clases**. Para esto, se utiliza un **análisis de varianza (ANOVA)** en lugar del estadístico t. Para esto se importa la función `f_oneway` de `scipy.stats`. Se extraen las observaciones cuya clase es 1 y los que son de clase 3 y se **estratifican** junto con los obtenidos previamente en **X1**, **X2**, **X3** y **X4**. Se guardan los **estádisticos f** y **p-values** en vectores y posteriormente en un dataframe y de nuevo se aplican las 3 metodologías de **bonferroni**, **holm** y **benjamini-hochberg** para obtener los genes que sean significativos en la diferencia de promedios entre las 4 clases. Se imprime la cantidad de genes significativos encontrados por cada metología y un resumen de estos.

In [98]:
from scipy.stats import f_oneway

X1 = X[y == 1]
X3 = X[y == 3]

f_stats = []
p_values = []

for col in X.columns:
  f_stat, p_val = f_oneway(X1[col], X2[col], X3[col], X4[col])
  f_stats.append(f_stat)
  p_values.append(p_val)

anova_results = pd.DataFrame({
    'gen': X.columns,
    'f_statistic': f_stats,
    'p_value': p_values
})

anova_results['p_bonferroni'] = multipletests(anova_results['p_value'], method='bonferroni')[1]
anova_results['p_holm'] = multipletests(anova_results['p_value'], method='holm')[1]
anova_results['p_bh'] = multipletests(anova_results['p_value'], method='fdr_bh')[1]

significativas_og = anova_results[anova_results['p_value'] < 0.05]
significativas_bonferroni = anova_results[anova_results['p_bonferroni'] < 0.05]
significativas_holm = anova_results[anova_results['p_holm'] < 0.05]
significativas_bh = anova_results[anova_results['p_bh'] < 0.05]

print(f"{len(significativas_og)} genes significativos originalmente:")
print(significativas_og[['gen', 'p_value']])

print(f"\n{len(significativas_bonferroni)} genes significativos con Bonferroni:")
print(significativas_bonferroni[['gen', 'p_bonferroni']])

print(f"\n{len(significativas_holm)} genes significativos con Holm:")
print(significativas_holm[['gen', 'p_holm']])

print(f"\n{len(significativas_bh)} genes significativos con Benjamini-Hochberg:")
print(significativas_bh[['gen', 'p_bh']])

1303 genes significativos originalmente:
        gen       p_value
0        X1  3.839240e-20
1        X2  1.977997e-13
2        X3  5.004749e-07
8        X9  8.500660e-05
11      X12  2.418413e-02
...     ...           ...
2299  X2300  4.569746e-05
2300  X2301  5.730919e-09
2301  X2302  1.665371e-02
2302  X2303  1.836660e-14
2303  X2304  4.752118e-06

[1303 rows x 2 columns]

404 genes significativos con Bonferroni:
        gen  p_bonferroni
0        X1  8.860966e-17
1        X2  4.565218e-10
2        X3  1.155096e-03
16      X17  1.996535e-03
28      X29  1.832080e-05
...     ...           ...
2293  X2294  9.538627e-03
2298  X2299  8.740287e-03
2300  X2301  1.322696e-05
2302  X2303  4.239011e-11
2303  X2304  1.096789e-02

[404 rows x 2 columns]

412 genes significativos con Holm:
        gen        p_holm
0        X1  8.837930e-17
1        X2  4.456428e-10
2        X3  1.031479e-03
16      X17  1.767297e-03
28      X29  1.706660e-05
...     ...           ...
2293  X2294  8.203715e-03


**4.** Con los datos obtenidos, se separan los datos en** entrenamiento y prueba** con `train_test_split`. Cabe aclarar, que al ser **mucho menos muestras que variables**, la proporción utilizada fue un **50/50** para **evitar el sobreajuste y resultados engañosos** al utilizar muy pocas muestras en test. Por la misma razón, **se redujo el número de variables** utilizando solamente los **10 genes más significativos** segun la metología de **benjamini-hochberg**. Una vez hecho esto, se generan **modelos de SVM (support vector machine)** utilizando la función `SVC` de `sklearn`. Se generan **3 kernels**: **lineal**, **polinomial de grado 3** y **radial**. Se ajustan los 3 modelos con los datos de entrenamiento y prueba y se obtienen las predicciones con los datos de test.

In [101]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

genes_modelo = anova_results.sort_values(by='p_bh').head(10)['gen']
print(f"genes a utilizar: \n {genes_modelo}")
x_gen_model = X[genes_modelo]

X_train, X_test, y_train, y_test = train_test_split(x_gen_model, y, test_size=0.5, random_state=42, stratify=y)

svm_linear = SVC(kernel = 'linear')
svm_poly = SVC(kernel='poly',degree=3)
svm_rbf = SVC(kernel='rbf')

svm_linear.fit(X_train, y_train)
svm_poly.fit(X_train, y_train)
svm_rbf.fit(X_train, y_train)

y_pred_linear = svm_linear.predict(X_test)
y_pred_poly = svm_poly.predict(X_test)
y_pred_rbf = svm_rbf.predict(X_test)

genes a utilizar: 
         gen  f_statistic       p_value  p_bonferroni        p_holm  \
1388  X1389    83.817537  1.772751e-24  4.091510e-21  4.089737e-21   
1954  X1955    84.364086  1.459035e-24  3.367454e-21  3.367454e-21   
1002  X1003    77.795622  1.618988e-23  3.736625e-20  3.733387e-20   
2049  X2050    69.230799  4.733702e-22  1.092539e-18  1.091118e-18   
245    X246    68.414042  6.633722e-22  1.531063e-18  1.528410e-18   
741    X742    65.572797  2.195548e-21  5.067325e-18  5.056347e-18   
0        X1    59.118264  3.839240e-20  8.860966e-17  8.837930e-17   
2161  X2162    56.987623  1.035143e-19  2.389109e-16  2.381863e-16   
1953  X1954    55.419914  2.182635e-19  5.037522e-16  5.020061e-16   
186    X187    54.615724  3.217900e-19  7.426914e-16  7.394735e-16   

              p_bh  
1388  2.045755e-21  
1954  2.045755e-21  
1002  1.245542e-20  
2049  2.731346e-19  
245   3.062126e-19  
741   8.445542e-19  
0     1.265852e-17  
2161  2.986387e-17  
1953  5.597246e-17  

**5.** Con los resultados obtenidos, se utiliza la función `classification_report` de `sklearn.metrics` para **analizar las metricas** de cada modelo y comparar los resultados.

In [100]:
from sklearn.metrics import classification_report

print("SVM con kernel lineal:")
print(classification_report(y_test, y_pred_linear))

print("\nSVM con kernel polinomial (grado 3):")
print(classification_report(y_test, y_pred_poly))

print("\nSVM con kernel radial:")
print(classification_report(y_test, y_pred_rbf))

SVM con kernel lineal:
              precision    recall  f1-score   support

           1       0.83      1.00      0.91         5
           2       0.93      0.93      0.93        15
           3       1.00      1.00      1.00         9
           4       1.00      0.92      0.96        13

    accuracy                           0.95        42
   macro avg       0.94      0.96      0.95        42
weighted avg       0.96      0.95      0.95        42


SVM con kernel polinomial (grado 3):
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         5
           2       1.00      0.80      0.89        15
           3       1.00      0.67      0.80         9
           4       0.68      1.00      0.81        13

    accuracy                           0.86        42
   macro avg       0.92      0.87      0.88        42
weighted avg       0.90      0.86      0.86        42


SVM con kernel radial:
              precision    recall  f1-score  

Con los resultados obtenidos, se puede observar que el modelo con **kernel radial tiene un mejor resultado en todas las métricas**. Aún así, **el modelo de kernel lineal muestra metricas bastante solidas** con buen accuracy y mejor precision en promedio por clase, además de ser el **más rápido computacionalmente**. Por esta razón, si lo que se busca es un **modelo robusto y preciso**, el **kernel radial** es el que cumple mejor la tarea. Pero si se busca un **equilibrio entre exactitud y rapidez**, el **kernel lineal** puede entregar muy buenos resultados utilizando menos poder computacional.